# Artificial Intelligence

## Convolutional Neural Networks

## Project: Write an Algorithm for a Room Image Category Identification App 

---

An imperfect solution will nonetheless create a fun user experience!

---

### Why We're Here 

In this notebook, you will make the first steps towards developing an algorithm that could be used as part of a mobile or web app.  At the end of this project, the code will accept any user-supplied image as input.  Given an image, it will estimate the category of the image. The image below displays potential sample output of your finished project. 

![Sample Image Output](hotel_images/420.deb5bf82_t.jpg)

In this real-world setting, you will need to piece together a series of models to perform different tasks;

### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Import Datasets
* [Step 1](#step1): Create a CNN to Classify Room Image Categories (from Scratch)
* [Step 2](#step2): Use a CNN to Classify Room Image Categories (using Transfer Learning)
* [Step 3](#step3): Create a CNN to Classify Room Image Categories (using Transfer Learning)
* [Step 4](#step4): Write the Algorithm
* [Step 5](#step5): Test the Algorithm

---
<a id='step0'></a>
## Step 0: Import Datasets

### Import Hotel Room Image Dataset

In the code cell below, we import a dataset of hotel room images.  We populate a few variables through the use of the `load_files` function from the scikit-learn library:
- `train_files`, `valid_files`, `test_files` - numpy arrays containing file paths to images
- `train_targets`, `valid_targets`, `test_targets` - numpy arrays containing onehot-encoded classification labels 
- `image_categories` - list of string-valued image category names for translating labels

In [ ]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    image_files = np.array(data['filenames'])
    image_targets = np_utils.to_categorical(np.array(data['target']), 228)
    return image_files, image_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('hotelImages/train')
valid_files, valid_targets = load_dataset('hotelImages/valid')
test_files, test_targets = load_dataset('hotelImages/test')

# load list of dog names
image_categories = [item[20:-1] for item in sorted(glob("hotelImages/train/*/"))]

# print statistics about the dataset
print('There are %d total room image categories.' % len(image_categories))
print('There are %s total room images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training room images.' % len(train_files))
print('There are %d validation room images.' % len(valid_files))
print('There are %d test room images.'% len(test_files))

In [ ]:
from keras.applications.resnet50 import ResNet50

# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

In [ ]:
from keras.applications.xception import Xception
Xception_model = Xception(weights='imagenet')

In [ ]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
from keras.applications.resnet50 import preprocess_input, decode_predictions

def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

In [ ]:
from keras.applications.xception import preprocess_input, decode_predictions

def Xception_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(Xception_model.predict(img))

In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, BatchNormalization, Activation
from keras.models import Sequential

num_classes=228

model = Sequential()

### TODO: Define your architecture.
model.add(Conv2D(16, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu', input_shape=(224,224,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
#model.add(Dropout(0.4))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Conv2D(32, kernel_size=(2, 2), strides=(1, 1), padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(32, kernel_size=(1, 1), strides=(1, 1), activation='relu'))
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))


model.summary()

### Compile the Model

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the Model

Train your model in the code cell below.  Use model checkpointing to save the model that attains the best validation loss.

You are welcome to [augment the training data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), but this is not a requirement. 

In [ ]:
from keras.callbacks import ModelCheckpoint  

### specify the number of epochs that you would like to use to train the model.
epochs = 3

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(test_tensors, test_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=2)

### Load the Model with the Best Validation Loss

In [ ]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

### Test the Model

Try out your model on the test dataset of dog images.  Ensure that your test accuracy is greater than 1%.

In [ ]:
# get index of predicted dog breed for each image in test set
image_type_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(image_type_predictions)==np.argmax(test_targets, axis=1))/len(image_type_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

---
<a id='step1'></a>
## Step 1: Use a CNN to Classify Room Image Categories

To reduce training time without sacrificing accuracy, we show you how to train a CNN using transfer learning.  In the following step, you will get a chance to use transfer learning to train your own CNN.

### Obtain Bottleneck Features

In [ ]:
bottleneck_features = np.load('bottleneck_features/RoomImageXceptionData.npz')
train_VGG16 = bottleneck_features['train']
valid_VGG16 = bottleneck_features['valid']
test_VGG16 = bottleneck_features['test']
print(train_VGG16.shape[1:])

### Model Architecture

The model uses the the pre-trained VGG-16 model as a fixed feature extractor, where the last convolutional output of VGG-16 is fed as input to our model.  We only add a global average pooling layer and a fully connected layer, where the latter contains one node for each dog category and is equipped with a softmax.

In [ ]:
VGG16_model = Sequential()
VGG16_model.add(GlobalAveragePooling2D(input_shape=train_VGG16.shape[1:]))
VGG16_model.add(Dense(228, activation='softmax'))

VGG16_model.summary()

### Compile the Model

In [ ]:
VGG16_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

### Train the Model

In [ ]:
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.VGG16.hdf5', 
                               verbose=1, save_best_only=True)

VGG16_model.fit(train_VGG16, train_targets, 
          validation_data=(valid_VGG16, valid_targets),
          epochs=20, batch_size=20, callbacks=[checkpointer], verbose=2)

### Load the Model with the Best Validation Loss

In [ ]:
VGG16_model.load_weights('saved_models/weights.best.VGG16.hdf5')

### Test the Model

Now, we can use the CNN to test how well it identifies breed within our test dataset of dog images.  We print the test accuracy below.

In [ ]:
# get index of predicted dog breed for each image in test set
VGG16_predictions = [np.argmax(VGG16_model.predict(np.expand_dims(feature, axis=0))) for feature in test_VGG16]

# report test accuracy
test_accuracy = 100*np.sum(np.array(VGG16_predictions)==np.argmax(test_targets, axis=1))/len(VGG16_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

### Predict Room Image Category with the Model

In [ ]:
from extract_bottleneck_features import *

def VGG16_predict_image_category(img_path):
    bottleneck_feature = extract_VGG16(path_to_tensor(img_path))
    predicted_vector = VGG16_model.predict(bottleneck_feature)
    return image_categories[np.argmax(predicted_vector)]

---
<a id='step4'></a>
## Step 4: Create a CNN to Classify Room Image Category (using Transfer Learning)

You will now use transfer learning to create a CNN that can identify dog breed from images.  Your CNN must attain at least 60% accuracy on the test set.

In Step 4, we used transfer learning to create a CNN using VGG-16 bottleneck features.  In this section, you must use the bottleneck features from a different pre-trained model.  To make things easier for you, we have pre-computed the features for all of the networks that are currently available in Keras:
- [VGG-19](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogVGG19Data.npz) bottleneck features
- [ResNet-50](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogResnet50Data.npz) bottleneck features
- [Inception](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogInceptionV3Data.npz) bottleneck features
- [Xception](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogXceptionData.npz) bottleneck features

The files are encoded as such:

    Dog{network}Data.npz
    
where `{network}`, in the above filename, can be one of `VGG19`, `Resnet50`, `InceptionV3`, or `Xception`.  Pick one of the above architectures, download the corresponding bottleneck features, and store the downloaded file in the `bottleneck_features/` folder in the repository.

### Obtain Bottleneck Features

In the code block below, extract the bottleneck features corresponding to the train, test, and validation sets by running the following:

    bottleneck_features = np.load('bottleneck_features/Dog{network}Data.npz')
    train_{network} = bottleneck_features['train']
    valid_{network} = bottleneck_features['valid']
    test_{network} = bottleneck_features['test']

In [ ]:
### Obtain bottleneck features from another pre-trained CNN.
bottleneck_features = np.load('bottleneck_features/RoomImageXceptionData.npz')
train_Xception = bottleneck_features['train']
valid_Xception = bottleneck_features['valid']
test_Xception = bottleneck_features['test']

In [ ]:
### TODO: Define your architecture.
print(train_Xception.shape)
print(valid_Xception.shape)
print(test_Xception.shape)
Xception_model = Sequential()
Xception_model.add(GlobalAveragePooling2D(input_shape=train_Xception.shape[1:]))
Xception_model.add(Dense(228, activation='softmax'))

Xception_model.summary()

### Compile the Model

In [ ]:
Xception_model.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['accuracy'])

### Train the Model

Train model in the code cell below.  Use model checkpointing to save the model that attains the best validation loss.  

You are welcome to [augment the training data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), but this is not a requirement. 

In [ ]:
### Train the model.
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.Xception.hdf5', 
                               verbose=1, save_best_only=True)

Xception_model.fit(train_Xception, train_targets, 
          validation_data=(valid_Xception, valid_targets),
          epochs=20, batch_size=50, callbacks=[checkpointer], verbose=2)

In [ ]:
### TODO: Load the model weights with the best validation loss.
Xception_model.load_weights('saved_models/weights.best.Xception.hdf5')

In [ ]:
### TODO: Calculate classification accuracy on the test dataset.
# get index of predicted dog breed for each image in test set
Xception_predictions = [np.argmax(Xception_model.predict(np.expand_dims(feature, axis=0))) for feature in test_Xception]

# report test accuracy
test_accuracy = 100*np.sum(np.array(Xception_predictions)==np.argmax(test_targets, axis=1))/len(Xception_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

In [ ]:
### TODO: Write a function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.
from extract_bottleneck_features import *

def detect_image_category(img_path):
    # extract bottleneck features
    bottleneck_feature = extract_Xception(path_to_tensor(img_path))
    # obtain predicted vector
    predicted_vector = Xception_model.predict(bottleneck_feature)
    # return image category that is predicted by the model
    return image_categories[np.argmax(predicted_vector)]

In [ ]:
def img_category_info(img_path):
    print(img_path)
    return 'Ah a room picture', detect_image_category(img_path)

In [ ]:
def test_sample_image(img_path):
    title, category = img_category_info(img_path)
    print(title)
    img = cv2.imread(img_path)
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(cv_rgb)
    plt.show()
    if category!=None:
        print(category)
    else:
        print('error')

---
<a id='step5'></a>
## Step 5: Test Algorithm

In this section, you will take your new algorithm for a spin!  What kind of dog does the algorithm think that __you__ look like?  If you have a dog, does it predict your dog's breed accurately?  If you have a cat, does it mistakenly think that your cat is a dog?

### Test Algorithm on Sample Images!

Test algorithm with at least six images on your computer.  

In [ ]:
## TODO: Execute your algorithm from Step 6 on
## at least 6 images on your computer.
## Feel free to use as many code cells as needed.
golden_retriver_image = 'my_images\IMG_1303.jpg'
test_sample_image(golden_retriver_image)


In [ ]:
another_golden_retriver_image = 'my_images\IMG_1296.jpg'
test_sample_image(another_golden_retriver_image)

In [ ]:
third_golden_retriver_image = 'my_images\\big_dog.PNG'
test_sample_image(third_golden_retriver_image)

In [ ]:
chihuahua_image = 'my_images\small_dog.PNG'
test_sample_image(chihuahua_image)

In [ ]:
another_chihuahua_image = 'my_images\IMG_1301.jpg'
test_sample_image(another_chihuahua_image)

In [ ]:
third_chihuahua_image = 'my_images\IMG_1295.jpg'
test_sample_image(third_chihuahua_image)

In [ ]:
human_dancer_image = 'my_images\IMG_1243.jpg'
test_sample_image(human_dancer_image)

In [ ]:
human_boy_image = 'my_images\IMG_0038.jpg'
test_sample_image(human_boy_image)

In [ ]:
human_man_image = 'my_images\IMG_0815.jpg'
test_sample_image(human_man_image)

In [ ]:
human_old_photo_image = 'my_images\old_photo.jpg'
test_sample_image(human_old_photo_image)

In [ ]:
sample_image = 'images\sample_human_output.png'
test_sample_image(sample_image)

In [ ]:
human_my_photo_image = 'my_images\IMG_1154.jpg'
test_sample_image(human_my_photo_image)